# In this part, we try to calculate the difficulty of a text, according to following formulation

### 含雾指数 = (每句平均字数 + 难字数 )  *0.7

In [1]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs
import jieba.posseg as pseg 


In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.714 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拆分句子
# 版本为python3，如果为python2需要在字符串前面加上u
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('([，！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)    
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")

In [23]:
file_path = './dataset/数据/一千零一夜.txt'
text = open(file_path, 'r',encoding="utf-8")
corpus = text.readlines()[1::2]
#corpus

In [24]:
arr = []
for c in corpus: 
    #print(type(c))
    #file_name='过年'
    #output_name =  file_name+' 句子长度.xlsx'
    #file_path = './dataset/raw_texts/'+file_name+'.txt'
    #text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
    df =pd.read_excel('./dataset/难字词库.xlsx') 
    diff_words = df['词语']
    result_words = []
    sent_arr = []
    words_arr = []
    sents = cut_sent(c)
    #print(sents)
    #print(block)  
    for sent in sents:
        words=list(seg_sentence(sent).split(' '))
        #print(words)
        sent_arr.append(len(sent))
        for w in words:
            #print(w)
            words_arr.append(w)
            if w in list(diff_words):
                #print(w)     
                if w not in result_words:
                    #print(w)
                    result_words.append(w)

    sents_mean = np.mean(sent_arr) #平均每句字数
    #print(sents_mean)
    diff_word_count=len(result_words) #文本难词数
    text_words_count=len(words_arr) #文本总词数
    diff_ratio = diff_word_count/text_words_count #文本难字率
    #print(diff_ratio)
    arr.append((sents_mean + diff_ratio))
    text_diff = (sents_mean + diff_ratio)*0.7  #含雾指数
    #print(':'+str(text_diff))
    

In [25]:
print(len(arr))

257


In [27]:
print(arr[0:101])

[9.258668346104054, 14.393126808270411, 16.20527972955058, 17.812223988694576, 13.216470588235293, 15.409027777777778, 13.937342767295599, 12.901008303677342, 12.601189351414439, 12.339203612479475, 16.163923440968297, 12.942268627610302, 11.543070728572577, 12.285415705637469, 13.793120393120393, 11.007236572825164, 13.63465981553703, 14.47623954599761, 12.00503379119236, 13.72484578824957, 14.361802206904711, 13.49278473728476, 12.113451230938798, 12.918711102467215, 14.53250996596101, 12.266090617037776, 15.110102403950904, 12.347065598157302, 14.35918600849951, 13.401923018408642, 13.02803738317757, 14.481760435571688, 14.567038539553753, 12.668961108439413, 15.928759148369926, 13.415584415584416, 11.962749169435217, 14.84828188701223, 14.677043620629165, 12.456920482743893, 14.830310122179485, 14.776530612244898, 12.817437533227007, 15.25188705619234, 15.741592312971859, 16.404871794871795, 22.679585798816568, 58.0, 15.60210897396252, 12.711130338252579, 13.13740477464394, 12.1638

In [173]:
sents_mean = np.mean(sent_arr) #平均每句字数
sents_mean

11.611842105263158

In [174]:
diff_word_count=len(result_words) #文本难字数
diff_word_count

22

In [175]:
text_diff = (sents_mean + diff_word_count)*0.7  #含雾指数
text_diff

23.528289473684207

In [13]:
# test code 
#file_name='班主任'
#file_path = './dataset/raw_texts/'+file_name+'.txt'
#text = open(file_path, 'r',encoding="utf-8") #加载要处理的文件的路径
#df =pd.read_excel('./dataset/难字词库.xlsx') 
diff_words = df['词语']
result_words = []
sent_arr = []
words_arr = []
for c in corpus:
    #print(type(block))
    sents = cut_sent(c)
    #print(block)  
    for sent in sents:
        words=list(seg_sentence(sent).split(' '))
        #print(words)
        sent_arr.append(len(sent))
        for w in words:
            #print(w)
            words_arr.append(w)
            if w in list(diff_words):
                #print(w)     
                if w not in result_words:
                    #print(w)
                    result_words.append(w)

sents_mean = np.mean(sent_arr) #平均每句字数
#print(sents_mean)
diff_word_count=len(result_words) #文本难词数
text_words_count=len(words_arr) #文本总词数
diff_ratio = diff_word_count/text_words_count #文本难字率
#print(diff_ratio)
text_diff = (sents_mean + diff_ratio)*0.7  #含雾指数
print(':'+str(text_diff))


:74.67864638803137


## 笔画数

In [204]:
def get_stroke(c):
    # 如果返回 0, 则也是在unicode中不存在kTotalStrokes字段
    strokes = []
    with open(strokes_path, 'r') as fr:
        for line in fr:
            strokes.append(int(line.strip()))

    unicode_ = ord(c)

    if 13312 <= unicode_ <= 64045:
        return strokes[unicode_-13312]
    elif 131072 <= unicode_ <= 194998:
        return strokes[unicode_-80338]
    else:
        print("c should be a CJK char, or not have stroke in unihan data.")
        # can also return 0